# Load data

In [1]:
!git clone https://github.com/YardenAronson/Amazon_Recommendation.git

Cloning into 'Amazon_Recommendation'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (14/14), 2.63 MiB | 6.25 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
import pandas as pd
import numpy as np

In [6]:
ratings_data = pd.read_csv('/content/Amazon_Recommendation/rating.csv')
ratings_data.head()

,user_id,product_id,rating,rating_count
0,AG3D6O4STAQKAY2UVGEUV46KN35Q,B07JW9H4J1,4.2,24269.0
1,AHMY5CWJMMK5BJRBBSNLYT3ONILA,B07JW9H4J1,4.2,24269.0
2,AHCTC6ULH4XB6YHDY6PCH2R772LQ,B07JW9H4J1,4.2,24269.0
3,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,B07JW9H4J1,4.2,24269.0
4,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,B07JW9H4J1,4.2,24269.0


In [15]:
# Ensure proper types
ratings_data['rating'] = pd.to_numeric(ratings_data['rating'], errors='coerce')
ratings_data.dropna(inplace=True)

# Modeling

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
product_user_matrix = ratings_data.pivot_table(index='product_id', columns='user_id', values='rating').fillna(0)
product_user_matrix

user_id,AE22E2AXODSPNK3EBIHNGYS5LOSA,AE22MK2NXQD3ZARLIOL3SLD4GU6A,AE22Y3KIS7SE6LI3HE2VS6WWPU4Q,AE23RS3W7GZO7LHYKJU6KSKVM4MQ,AE23WGYTUMB5R6JJMBU4V43JIW7Q,AE242TR3GQ6TYC6W4SJ5UYYKBTYQ,AE243IWFZJ3BB6E6WMUG52DHWJVA,AE24UFIVBSESSEV7UALTKP7K5Z4A,AE26ZTVZB6CB4VML6JSTYTL2QG6A,AE27PFEMMMJS44GT27KPL6VUOQUQ,...,AHZSWCFTWESY2N2RK4BVWBMM4TZQ,AHZTLZNYRSIEOCNNYMQCXWTWA62Q,AHZWJCVEIEI76H2VGMUSN5D735IQ,AHZWXUWE3RGLDH4JJUK3HT3VMBJA,AHZXKAGAJPIMZJD5XJ5QUIYR3ORA,AHZXNS63DN6MZDH3WSKYRLWSG3DA,AHZYN7O73FJWRPUQGZM5BEAZ3A4A,AHZZFBL24XXVLW6H44MOB6LBHH5A,AHZZISWHBQV4TL3MKOJTH22IUNIQ,AHZZXJWETMZR7SH7C22KVUT7VHAA
product_id,,,,,,,,,,,,,,,,,,,,,
B002PD61Y4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B002SZEOLG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B003B00484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B003L62T7W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B004IO5BMQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0BPBXNQQT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0BPCJM7TB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0BPJBTB3F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from scipy.sparse.linalg import svds
import numpy as np

# Convert the pivot table to a sparse matrix
from scipy.sparse import csr_matrix
product_user_sparse_matrix = csr_matrix(product_user_matrix.values)

# Perform SVD
U, sigma, Vt = svds(product_user_sparse_matrix, k=50)
sigma = np.diag(sigma)

# Compute predicted ratings
predicted_ratings = np.dot(np.dot(U, sigma), Vt)
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=product_user_matrix.index, columns=product_user_matrix.columns)


In [27]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute product similarity matrix
product_similarity_matrix = cosine_similarity(predicted_ratings_df)
product_similarity_df = pd.DataFrame(product_similarity_matrix, index=product_user_matrix.index, columns=product_user_matrix.index)


In [31]:
import pandas as pd

def recommend_products_based_on_product(product_id, product_similarity_df, top_n=10):
    """
    Recommends products similar to the given product based on a product similarity matrix.

    Parameters:
    - product_id: The ID of the product for which recommendations are to be made.
    - product_similarity_df: DataFrame where columns and index are product IDs, and cell values are similarity scores.
    - top_n: Number of top similar products to return (default is 10).

    Returns:
    - A DataFrame containing the top `n` recommended products and their similarity scores.
    """

    # Check if the product_id is in the similarity matrix
    if product_id not in product_similarity_df.index:
        raise ValueError(f"Product ID {product_id} not found in the dataset.")

    # Get the similarity scores for the given product
    similar_products = product_similarity_df.loc[product_id]

    # Sort products based on similarity scores
    similar_products = similar_products.sort_values(ascending=False)

    # Filter out the original product from recommendations
    similar_products = similar_products[similar_products.index != product_id]

    # Get the top n recommendations
    top_recommendations = similar_products.head(top_n)

    # Convert to DataFrame for a structured output
    recommendations_df = top_recommendations.reset_index()
    recommendations_df.columns = ['Product ID', 'Similarity Score']

    return recommendations_df


In [32]:
# Example usage
product_id = 'B08CF3B7N1'  # example product ID
recommendations = recommend_products_based_on_product(product_id, product_similarity_df, 5)
print(recommendations)

   Product ID  Similarity Score
0  B01KCSGBU2      1.208947e-15
1  B01D5H8ZI8      1.200294e-15
2  B079S811J3      1.199612e-15
3  B012ELCYUG      1.189378e-15
4  B082T6V3DT      1.187318e-15


In [30]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Calculate RMSE (Root Mean Squared Error)
def calculate_rmse():
    actual_ratings = product_user_sparse_matrix.toarray().flatten()
    predicted_ratings_flat = predicted_ratings.flatten()

    # Consider only the non-zero entries
    non_zero_indices = actual_ratings.nonzero()
    rmse = sqrt(mean_squared_error(actual_ratings[non_zero_indices], predicted_ratings_flat[non_zero_indices]))
    print(f'RMSE: {rmse}')

calculate_rmse()


RMSE: 3.813599601258082


In [33]:
product_similarity_df.to_csv('product_similarity.csv')